In [18]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
from pyspark.sql import SQLContext 
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf,col,when
from pyspark.sql.functions import to_timestamp,date_format
import numpy as np
import pandas as pd
from pyspark.mllib.stat import Statistics 
from math import sqrt 
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

In [2]:
spark = SparkSession.builder.master("local[1]").appName("Recommendations").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/18 20:18:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
transaction = spark.read.option("header",True) \
              .csv("transactions_train.csv")
transaction.printSchema()

root
 |-- t_dat: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- sales_channel_id: string (nullable = true)



In [4]:
transaction.show()

+----------+--------------------+----------+--------------------+----------------+
|     t_dat|         customer_id|article_id|               price|sales_channel_id|
+----------+--------------------+----------+--------------------+----------------+
|2018-09-20|000058a12d5b43e67...|0663713001|0.050830508474576264|               2|
|2018-09-20|000058a12d5b43e67...|0541518023| 0.03049152542372881|               2|
|2018-09-20|00007d2de826758b6...|0505221004| 0.01523728813559322|               2|
|2018-09-20|00007d2de826758b6...|0685687003|0.016932203389830508|               2|
|2018-09-20|00007d2de826758b6...|0685687004|0.016932203389830508|               2|
|2018-09-20|00007d2de826758b6...|0685687001|0.016932203389830508|               2|
|2018-09-20|00007d2de826758b6...|0505221001|0.020322033898305086|               2|
|2018-09-20|00083cda041544b2f...|0688873012| 0.03049152542372881|               1|
|2018-09-20|00083cda041544b2f...|0501323011|0.053372881355932204|               1|
|201

In [6]:
df1 =  transaction.withColumn('t_dat', transaction['t_dat'].cast('string'))
df1 = df1.groupby('customer_id', 'article_id').count()
df1.show(10)

+--------------------+----------+-----+
|         customer_id|article_id|count|
+--------------------+----------+-----+
|00d781e94d9a533dc...|0616708004|    1|
|0164840429884bbb4...|0578589001|    1|
|01c8cb2f730cd7253...|0537688014|    1|
|0247a4cfbe56ac5e1...|0618840005|    1|
|0745af85843f2d073...|0674492001|    1|
|0849f9bb66e8d4695...|0684010001|    1|
|09e1f56b250e4a78b...|0525932002|    1|
|0a8e062f9162d6bab...|0643217002|    1|
|0ae14ec796c207d31...|0522398023|    1|
|0bb8f46440abf3d28...|0637515010|    1|
+--------------------+----------+-----+
only showing top 10 rows



In [7]:
userId_count = df1.groupBy("customer_id").count().orderBy('count', ascending=False)
userId_count.show()

+--------------------+-----+
|         customer_id|count|
+--------------------+-----+
|be1981ab818cf4ef6...| 1346|
|cd04ec2726dd58a8c...| 1121|
|49beaacac0c7801c2...| 1088|
|b4db5e5259234574e...| 1068|
|55d15396193dfd458...|  972|
|a65f77281a528bf5c...|  906|
|6cc121e5cc202d2bf...|  877|
|a76cf5ea515d09f22...|  874|
|d80ed4ababfa96812...|  859|
|689f4eda82fdf3d9b...|  840|
|c140410d72a41ee5e...|  836|
|3493c55a7fe252c84...|  834|
|03d0011487606c37c...|  807|
|1df07f916d7f64845...|  784|
|898ede9fb639eb2ae...|  765|
|0d4fb6fb46dfe2759...|  765|
|6980555e2eaa19a3f...|  742|
|77db96923d20d4053...|  723|
|e238725cbff3774b7...|  719|
|8df45859ccd71ef1e...|  718|
+--------------------+-----+
only showing top 20 rows



In [8]:
df_article = spark.read.option("header",True) \
              .csv("df_article.csv")
df_article.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- article_id: string (nullable = true)
 |-- product_code: string (nullable = true)
 |-- prod_name: string (nullable = true)
 |-- product_type_no: string (nullable = true)
 |-- product_type_name: string (nullable = true)
 |-- product_group_name: string (nullable = true)
 |-- graphical_appearance_no: string (nullable = true)
 |-- graphical_appearance_name: string (nullable = true)
 |-- colour_group_code: string (nullable = true)
 |-- colour_group_name: string (nullable = true)
 |-- perceived_colour_value_id: string (nullable = true)
 |-- perceived_colour_value_name: string (nullable = true)
 |-- perceived_colour_master_id: string (nullable = true)
 |-- perceived_colour_master_name: string (nullable = true)
 |-- department_no: string (nullable = true)
 |-- department_name: string (nullable = true)
 |-- index_code: string (nullable = true)
 |-- index_name: string (nullable = true)
 |-- index_group_no: string (nullable = true)
 |-- index_group_name

In [26]:
df_article.columns

['_c0',
 'article_id',
 'product_code',
 'prod_name',
 'product_type_no',
 'product_type_name',
 'product_group_name',
 'graphical_appearance_no',
 'graphical_appearance_name',
 'colour_group_code',
 'colour_group_name',
 'perceived_colour_value_id',
 'perceived_colour_value_name',
 'perceived_colour_master_id',
 'perceived_colour_master_name',
 'department_no',
 'department_name',
 'index_code',
 'index_name',
 'index_group_no',
 'index_group_name',
 'section_no',
 'section_name',
 'garment_group_no',
 'garment_group_name',
 'detail_desc',
 't_dat',
 'customer_id',
 'price',
 'sales_channel_id',
 'Total customer']